In [13]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.losses import BinaryCrossentropy

In [14]:
G_train_df      = pd.read_csv ('../data/preprocessed_data/G_train.csv')
T_train_df      = pd.read_csv ('../data/preprocessed_data/T_train.csv')
G_T_train_df    = pd.read_csv ('../data/preprocessed_data/balanced_G_T_train.csv')

G_cv_df         = pd.read_csv ('../data/preprocessed_data/G_cv.csv')
T_cv_df         = pd.read_csv ('../data/preprocessed_data/T_cv.csv')
G_T_cv_df       = pd.read_csv ('../data/preprocessed_data/G_T_cv.csv')


In [15]:
random_state = 13
frac = 0.1

G_train_df      = G_train_df.sample(frac = frac, random_state=random_state)
T_train_df      = T_train_df.sample(frac = frac, random_state=random_state)
G_T_train_df    = G_T_train_df.sample(frac = frac, random_state=random_state)

In [16]:
y_train = G_T_train_df.drop(columns = ['group ID','technique ID' ]).values
y_train.dtype

# G_train = sampled_G_train_df.drop(columns = ids)
G_train = G_train_df.drop(columns = 'group ID').values

# T_train = sampled_T_train_df.drop(columns = ids)
T_train = T_train_df.drop(columns = 'technique ID').values

In [17]:
G_train_tf = tf.convert_to_tensor(G_train)
T_train_tf = tf.convert_to_tensor(T_train)
y_train_tf = tf.convert_to_tensor(y_train)
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf))

batch_size = 32


train_dataset = train_dataset.shuffle(buffer_size=len(G_train_tf))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [18]:
y_cv = G_T_cv_df.drop(columns = ['group ID','technique ID' ]).values
y_cv.dtype

# G_cv = sampled_G_cv_df.drop(columns = ids)
G_cv = G_cv_df.drop(columns = 'group ID').values

# T_cv = sampled_T_cv_df.drop(columns = ids)
T_cv = T_cv_df.drop(columns = 'technique ID').values

In [19]:
G_cv_tf = tf.convert_to_tensor(G_cv)
T_cv_tf = tf.convert_to_tensor(T_cv)
y_cv_tf = tf.convert_to_tensor(y_cv)
cv_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_cv_tf, 'input_Technique': T_cv_tf}, y_cv_tf))
cv_dataset = cv_dataset.batch(batch_size)

In [20]:
import sys
sys.path.append('../models') 

from model_v0_2 import ContentBasedFiltering

In [21]:
# input shapes config
num_G_features = G_train.shape[1]  # remove Group ID during training
num_T_features = T_train.shape[1]   # remove Movie ID during training

model1 = ContentBasedFiltering (
    num_G_features=num_G_features, 
    num_T_features=num_T_features,
    Group_NN_width = 128,
    Group_NN_depth = 2,
    Technique_NN_width = 128,
    Technique_NN_depth = 2,)

In [22]:
model1.model.summary()

Model: "recsysNN_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_Group (InputLayer)       [(None, 463)]        0           []                               
                                                                                                  
 input_Technique (InputLayer)   [(None, 54)]         0           []                               
                                                                                                  
 Group_NN (Sequential)          (None, 32)           80032       ['input_Group[0][0]']            
                                                                                                  
 Technique_NN (Sequential)      (None, 32)           27680       ['input_Technique[0][0]']        
                                                                                     

In [23]:
epochs = 50
history = model1.train (
    train_dataset,
    cv_dataset,
    epochs= epochs
)

Epoch 1/50


403/403 [==============================] - 3s 5ms/step - loss: 0.8021 - val_loss: 0.6882
Epoch 2/50
403/403 [==============================] - 2s 4ms/step - loss: 0.6504 - val_loss: 0.5702
Epoch 3/50
403/403 [==============================] - 2s 4ms/step - loss: 0.6219 - val_loss: 0.6234
Epoch 4/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6414 - val_loss: 0.7176
Epoch 5/50
403/403 [==============================] - 2s 4ms/step - loss: 0.6616 - val_loss: 0.5954
Epoch 6/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6232 - val_loss: 0.5986
Epoch 7/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6204 - val_loss: 0.6119
Epoch 8/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6152 - val_loss: 0.6809
Epoch 9/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6173 - val_loss: 0.5768
Epoch 10/50
403/403 [==============================] - 2s 5ms/step - loss: 0.6147 - val_loss: 0.5670
Epoch 11/5